In [192]:
#spacy
import spacy
import en_core_web_lg
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc, Span
from spacy.matcher import Matcher, PhraseMatcher


#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
#import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines
import json
import os
import csv

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lewagonlew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/lewagonlew/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
PATH = os.path.dirname(os.path.dirname(os.getcwd()))
PATH

'/home/lewagonlew/code/marcorodrigues'

In [104]:
skills = pd.read_csv(PATH + "/skills_matcher/data/dictionaries/all_skills.csv")
skills.shape

(31402, 2)

In [284]:
new_skills = pd.read_csv(PATH + "/skills_matcher/data/dictionaries/new_dictionary.csv")
new_skills.shape

(2883, 2)

In [220]:
skill_pattern = []
with open('../data/skill_patterns.jsonl') as f:
    for line in f:
        skill_pattern.append(json.loads(line))

In [238]:
len(skill_patterns)

4251

In [246]:
(new_skills["Label"] == 'KNOWLEDGE').sum()

500

In [ ]:
for i in range(0, len(new_skills)):
    pass

(2883, 2)

In [275]:
skills

,Label,Skill
0,SKILL,Emergency Managers Weather Information Network...
1,SKILL,Federal Emergency Management Information Syste...
2,SKILL,SoftRisk Technologies SoftRisk SQL
3,SKILL,Sungard Assurance
4,SKILL,Digital Engineering Corporation E-MAPS
...,...,...
2117,SKILL,Graphing software
2118,SKILL,Meter reading software
2119,SKILL,Audio calibration software
2120,SKILL,IntelliTrack DMS Check In-Out


In [277]:
list1 = []
for i in range(0, len(skills['Skill'])):
    dict1 = {}
    dict1['label'] = 'SKILL'
    skill = skills['Skill'][i]
    sub_dict = {}
    sub_dict['TEXT'] = skill
    dict1['pattern'] = [sub_dict]
    list1.append(dict1)
    

In [285]:
new_skills

,Label,Skill
0,SKILL,1CadCam Unigraphics
1,SKILL,3D graphics software
2,SKILL,3DGrid HIPAA Checkup
3,SKILL,3M Electronic Monitoring
4,SKILL,3M Post-it App
...,...,...
2878,MIN_EXP,3 to 4 years
2879,MIN_EXP,4 to 5 years
2880,MIN_EXP,5 to 6 years
2881,MIN_EXP,10 years


In [294]:
new_list = []
for i in range(0, len(new_skills)):
    if new_skills["Label"][i] == "SKILL":
        dict1 = {}
        dict1['label'] = 'SKILL|'+ new_skills["Skill"][i].replace(' ', '-')
        skill = new_skills['Skill'][i].lower().split(' ')
        sub_dict = {}
        sub_list = []

        for item in skill:
            sub_dict['LOWER'] = item
            sub_list.append(sub_dict.copy())
            dict1['pattern'] = sub_list
        new_list.append(dict1)
        
    if new_skills["Label"][i] == "KNOWLEDGE":
        dict1 = {}
        sub_dict = {}
        sub_list = []
        dict1['label'] = 'KNOWLEDGE'
        
        sub_dict['LOWER'] = new_skills['Skill'][i]
        sub_list.append(sub_dict.copy())
        dict1['pattern'] = sub_list
        new_list.append(dict1)
        
    if new_skills["Label"][i] == "MIN_EXP":
        dict1 = {}
        sub_dict = {}
        sub_list = []
        dict1['label'] = 'MIN_EXP'
        
        sub_dict['LOWER'] = new_skills['Skill'][i]
        sub_list.append(sub_dict.copy())
        dict1['pattern'] = sub_list
        new_list.append(dict1)

new_list[-1]

{'label': 'MIN_EXP', 'pattern': [{'LOWER': ' 1+ years'}]}

In [302]:
new_list[-1]

{'label': 'MIN_EXP', 'pattern': [{'LOWER': ' 1+ years'}]}

In [117]:
import ujson
from pathlib import Path

def read_jsonl(file_path):
    """Read a .jsonl file and yield its contents line by line.
    file_path (unicode / Path): The file path.
    YIELDS: The loaded JSON contents of each line.
    """

    with Path(file_path).open('r', encoding='utf8') as f:
        for line in f:
            try:  # hack to handle broken jsonl
                yield ujson.loads(line.strip())
            except ValueError:
                continue


def write_jsonl(file_path, lines):
    """Create a .jsonl file and dump contents.
    file_path (unicode / Path): The path to the output file.
    lines (list): The JSON-serializable contents of each line.
    """
    data = [ujson.dumps(line, escape_forward_slashes=False) for line in lines]
    Path(file_path).open('w', encoding='utf-8').write('\n'.join(data))

In [239]:
skill_json_path = PATH + '/skills_matcher/data/dictionaries/skill_patterns.jsonl'

write_jsonl(skill_json_path, skill_patterns)

In [301]:
new_dict_json_path = PATH + '/skills_matcher/data/dictionaries/new_dict.jsonl'

write_jsonl(new_dict_json_path, new_list)

In [8]:
##renaming columns for entity_ruler and saving as new csv
#skills = skills.rename(columns={'Label':'label', 'Skill':'pattern'})
#skills.to_csv(PATH + "/skills_matcher/data/dictionaries/all_skills_new_labels.csv", index = False)

In [9]:
#open csv and turning it into a list of dictionaries
#with open(PATH + '/skills_matcher/data/dictionaries/all_skills_new_labels.csv') as f:
#    skills_dict = [{k: str(v) for k, v in row.items()}
#        for row in csv.DictReader(f, skipinitialspace=True)]

In [54]:
#skills_dict[0:10]

In [12]:
#job description data

data = pd.read_csv(PATH + '/skills_matcher/data/cleaned_data.csv')
data = data.drop(['Unnamed: 0'], axis = 1)
data = data.dropna()
data = data.drop_duplicates()
data.head()

,ISCO,major_job,job,position,location,description
0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...
1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,Geneva,[' Be in charge of the development of applicat...
2,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,Geneva,[' Contribute to the maintenance and developme...
3,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Medical Devices Physicist,"Newton, Cambridgeshire",[' Agency: Newton Colmore Consulting Reference...
4,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Fluidics Physicist,Cambridge,[' Agency: Newton Colmore Consulting Reference...


In [13]:
data = data.drop(columns = ["ISCO", "major_job", "location"])

In [14]:
data.head()

,job,position,description
0,physicist,Accelerator Physicist id54315,[' You have an academic degree in physics or e...
1,physicist,Applied Physicist (Computing) (EP-LBC-2021-125...,[' Be in charge of the development of applicat...
2,physicist,Accelerator Physicist (BE-ABP-LNO-2021-122-LD)...,[' Contribute to the maintenance and developme...
3,physicist,Medical Devices Physicist,[' Agency: Newton Colmore Consulting Reference...
4,physicist,Fluidics Physicist,[' Agency: Newton Colmore Consulting Reference...


In [26]:
#mixing job description and taking smaller sample for faster computation

data = data.reindex(np.random.permutation(data.index))
data = data.copy().iloc[
    0:200,
]
data["description"][3516]

'[\' Lead on documenting and understanding the existing SQL Databases Develop and Implement group wide Data Governance policies Act as day-to-day Data/BI System support to all department Support Manage in seeking ways to automate and standardise reporting across the Group Build robust Reporting Suites that measure financial and operational performance (Power BI) Partner Department Heads and Finance BPs, to understand performance drivers and standardise KPIs Support Manager in prioritising System Development requests Act as the day-to-day \\\'Data/Insight\\\' support for Bailey UK teams Work with Manager to develop and introduce Data Governance policies Document and develop understanding of existing system SQL databases and tables Work with Manager and Heads of departments to develop a data dictionary Develop "one source of the truth" reporting capability for the whole business, including of assessment of potential system requirements Provide commercial and finance reporting support for

# different spacy models

In [189]:
# spacy english model - change to small (en_core_web_lg) for faster computation - trained with skill_pattern.jsonl

nlp_ms = spacy.load("en_core_web_sm")
skill_pattern_path =  "../data/skill_patterns.jsonl"

ruler = nlp_ms.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp_ms.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [303]:
#2nd model trained with our skill_list

nlp_2 = spacy.load('en_core_web_sm')

ruler_2 = nlp_2.add_pipe("entity_ruler")

ruler_2.from_disk(new_dict_json_path)

nlp_2.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [174]:
skill_list[1]["pattern"]

[{'LOWER': 'federal'},
 {'LOWER': 'emergency'},
 {'LOWER': 'management'},
 {'LOWER': 'information'},
 {'LOWER': 'system'},
 {'LOWER': 'femis'}]

In [179]:
skills["Skill"][5:10]

5                         Alert Technologies OpsCenter
6            Emergency Services Integrators ESi WebEOC
7    National Center for Crisis and Continuity Coor...
8                                        Relex Weibull
9                         Microsoft Dynamics Marketing
Name: Skill, dtype: object

## cleaning and skill extraction functions

In [22]:
def get_skills(text):
    doc = nlp_2(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [23]:
#cleaning data function - call with TRUE value if lowering/stopword removiing/punctuation removing/ lemmatizing should be done
def clean_data(data, lower = False, stopw=False, punctuation=False, lemmatize=False):
    data = data.dropna()
    
    # digits
    data['description'] = data['description'].str.replace('\d+', '')
        
    if lower == True:
            ## Lower case
            data['description'] = data['description'].apply(lambda x: " ".join(x.lower()for x in x.split()))
            
    ## remove tabulation and punctuation
    if punctuation == True:
            data['description'] = data['description'].str.replace('[^\w\s]',' ')
            
    ## stopwords
    if stopw == True:
            stop = stopwords.words('english')
            data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
            
    ## lemmatizing
    if lemmatize == True:
            lemmatizer = WordNetLemmatizer()
            data['description'] = data['description'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

    ##depending on data frame, drop more columns, re-index
    ##clean_data.drop(columns = "position")
    
    return data

In [24]:
clean_data = clean_data(data, True, True, True, True)

In [25]:
clean_data.head()

,job,position,description
2640,recruitment officer,Recruitment Officer,involved whole lifecycle recruitment including...
2528,accountant,GL Accountant,significant support preparation monthly quarte...
52,biochemist,Biochemist,agency newton colmore consulting reference bmd...
3697,ux designer,UX Designer,innovative dynamic customer centric workplace ...
3056,video editor,Established Video Editor,editing ugc original content snapchat instagra...


In [27]:
clean_data.shape

(200, 3)

In [123]:
clean_data["skills"] = clean_data["description"].str.lower().apply(get_skills)
clean_data["skills"] = clean_data["skills"].apply(unique_skills)


,job,position,description,skills
2640,recruitment officer,Recruitment Officer,involved whole lifecycle recruitment including...,[]
2528,accountant,GL Accountant,significant support preparation monthly quarte...,[]
52,biochemist,Biochemist,agency newton colmore consulting reference bmd...,[]
3697,ux designer,UX Designer,innovative dynamic customer centric workplace ...,[]
3056,video editor,Established Video Editor,editing ugc original content snapchat instagra...,[]
...,...,...,...,...
1467,civil engineer,Principal Civil Engineer,working within small team providing technical ...,[]
1356,drama teacher,Music & Drama Teacher,hold working towards uk recognized teaching qu...,[]
183,process chemist,Senior Process Chemist / Agrochemicals,stable growing business great opportunity exce...,[]
3368,web developer,Web Developer,considerable experience developer technologist...,[]


In [137]:
clean_data.head(100)

,job,position,description,skills
2640,recruitment officer,Recruitment Officer,involved whole lifecycle recruitment including...,[]
2528,accountant,GL Accountant,significant support preparation monthly quarte...,[]
52,biochemist,Biochemist,agency newton colmore consulting reference bmd...,[]
3697,ux designer,UX Designer,innovative dynamic customer centric workplace ...,[]
3056,video editor,Established Video Editor,editing ugc original content snapchat instagra...,[]
...,...,...,...,...
4304,attendant,Room Attendant (Housekeeping),make sure guest room look well presented perfo...,[]
2901,cashier,Finance Assistant,generous holiday allowance day pro rata bank h...,[]
4113,product manager,Product Manager,assist generating vision idea social mobile ga...,[]
1459,nurse,Registered Nurse,location falkirk full time privately owned gro...,[]


In [140]:
clean_data['description'][4113]

'assist generating vision idea social mobile game market position feature viral engaging highly monetizeable go idea detailed concept test user decide implementation create wireframe screen mock ups user flow new product feature content manage team artist engineer prioritize task manage roadmap ensure implementation define measure analyse kpi base priority decision next step metric performing systematic b test split testing constantly optimise game post launch even virality engagement monetization bc mobile experience numerical skill gaming environment must obsessed detail good logical sound analytical numerical capability love product create passionate game think user point view want get better time ideal candidate turn difficult concept something magically simple user product manager creative mindset communicate clearly straight point experience product management game design mobile gaming environment infamous rolling sleeve get thing done care lot people playing game team company wo

In [138]:
unique_skills(get_skills(clean_data['description'][1459]))

[]

In [ ]:
text = 'peoplenet facebook go has a creative mindeste '

In [304]:
sent = nlp_2(clean_data["description"][4113])
displacy.render(sent, style="ent", jupyter=True)

In [305]:
sent = nlp_ms(clean_data["description"][4113])
displacy.render(sent, style="ent", jupyter=True)

# Same approach with different data (resume.csv from Kaggle)

In [ ]:
#resume data from kaggle - for tryout

df = pd.read_csv(PATH + "/skills_matcher/data/Resume/Resume.csv")
df.head()


In [ ]:
df = df.reindex(np.random.permutation(df.index))
df = df.copy().iloc[
    0:200,
]
df.head()

In [ ]:
#different cleaning function

clean = []
for i in range(df.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        df["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [ ]:
df["Clean_Resume"] = clean
df["skills"] = df["Clean_Resume"].str.lower().apply(get_skills)
df["skills"] = df["skills"].apply(unique_skills)
df.head()

In [ ]:
df["skills"]

In [ ]:
sent = nlp(df["Clean_Resume"].iloc[10])
displacy.render(sent, style="ent", jupyter=True)

In [ ]:
test = df["Resume_str"].iloc[4]
print(test)
unique_skills(get_skills(test))

## adding Job Category as Entity

In [ ]:
df.Category.unique()

In [ ]:
patterns = df.Category.unique()
for a in patterns:
    ruler.add_patterns([{"label": "Job-Category", "pattern": a}])

In [ ]:
#options= {"ents": "Job-Category", "colors": "#ff3232"},{"ents": "SKILL", "colors": "#56c426"}
colors = {
    "Job-Category": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
    "ORG": "#ffd966",
    "PERSON": "#e06666",
    "GPE": "#9fc5e8",
    "DATE": "#c27ba0",
    "ORDINAL": "#674ea7",
    "PRODUCT": "#f9cb9c",
}
options = {
    "ents": [
        "Job-Category",
        "SKILL",
        "ORG",
        "PERSON",
        "GPE",
        "DATE",
        "ORDINAL",
        "PRODUCT",
    ],
    "colors": colors,
}
sent = nlp_2(df["Resume_str"].iloc[99])
displacy.render(sent, style="ent", jupyter=True, options=options)